In [1]:
!pip install ucimlrepo
!pip install --upgrade xgboost


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ucimlrepo import fetch_ucirepo


diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)


X = diabetes_130_us_hospitals_for_years_1999_2008.data.features
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets


print(diabetes_130_us_hospitals_for_years_1999_2008.metadata)


print(diabetes_130_us_hospitals_for_years_1999_2008.variables)


{'uci_id': 296, 'name': 'Diabetes 130-US Hospitals for Years 1999-2008', 'repository_url': 'https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008', 'data_url': 'https://archive.ics.uci.edu/static/public/296/data.csv', 'abstract': 'The dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. Each row concerns hospital records of patients diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days. The goal is to determine the early readmission of the patient within 30 days of discharge.\nThe problem is important for the following reasons. Despite high-quality evidence showing improved clinical outcomes for diabetic patients who receive various preventive and therapeutic interventions, many patients do not receive them. This can be partially attributed to arbitrary diabetes management in hospital environments, which fail to attend to glycemic control. Failure to provide pro

C:\Users\ranul\AppData\Roaming\Python\Python310\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [4]:
dataframe = X.join(y)
dataframe.head()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 48 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      99493 non-null   object
 1   gender                    101766 non-null  object
 2   age                       101766 non-null  object
 3   weight                    3197 non-null    object
 4   admission_type_id         101766 non-null  int64 
 5   discharge_disposition_id  101766 non-null  int64 
 6   admission_source_id       101766 non-null  int64 
 7   time_in_hospital          101766 non-null  int64 
 8   payer_code                61510 non-null   object
 9   medical_specialty         51817 non-null   object
 10  num_lab_procedures        101766 non-null  int64 
 11  num_procedures            101766 non-null  int64 
 12  num_medications           101766 non-null  int64 
 13  number_outpatient         101766 non-null  int64 
 14  numb

In [6]:
dataframe.describe()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [7]:
print(dataframe.isnull().sum())

race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [8]:
dataframe['race'].fillna(dataframe['race'].mode()[0], inplace=True)

dataframe.drop(columns=['weight'], inplace=True)

dataframe.drop(columns=['payer_code'], inplace=True)

dataframe.drop(columns=['max_glu_serum'], inplace=True)

dataframe['medical_specialty'].fillna('Unknown', inplace=True)

dataframe['A1Cresult'].fillna('Not Tested', inplace=True)

dataframe['diag_1'].fillna('Unknown', inplace=True)
dataframe['diag_2'].fillna('Unknown', inplace=True)
dataframe['diag_3'].fillna('Unknown', inplace=True)

print(dataframe.isnull().sum())

C:\Users\ranul\AppData\Local\Temp\ipykernel_21272\1196355763.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe['race'].fillna(dataframe['race'].mode()[0], inplace=True)
C:\Users\ranul\AppData\Local\Temp\ipykernel_21272\1196355763.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

race                        0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide

In [9]:
diag_mapping = {
    '250': 'Diabetes',
    '276': 'Fluid Disorders',
    '414': 'Coronary Issues',
    '428': 'Heart Failure',
    '518': 'Respiratory Issues',
    '999': 'Medical Complications',
}


def map_diagnosis(code):
    if isinstance(code, str):
        prefix = code.split('.')[0]  
        return diag_mapping.get(prefix, 'Other') 
    return 'Other'


dataframe['diag_1'] = dataframe['diag_1'].apply(map_diagnosis)
dataframe['diag_2'] = dataframe['diag_2'].apply(map_diagnosis)
dataframe['diag_3'] = dataframe['diag_3'].apply(map_diagnosis)

In [10]:
dataframe.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,41,0,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,Unknown,59,0,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,Unknown,11,5,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,Unknown,44,1,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,Unknown,51,0,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [11]:
dataframe.drop_duplicates(inplace=True)

In [12]:
dataframe['readmitted'] = dataframe['readmitted'].apply(lambda x: 0 if x == 'No' else (1 if x == '<30' else 2))
dataframe['change'] = dataframe['change'].apply(lambda x: 1 if x == 'change' else 0)
dataframe['diabetesMed'] = dataframe['diabetesMed'].apply(lambda x: 1 if x == 'yes' else 0)


dataframe['repaglinide'] = dataframe['repaglinide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['nateglinide'] = dataframe['nateglinide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['chlorpropamide'] = dataframe['chlorpropamide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['glimepiride'] = dataframe['glimepiride'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['acetohexamide'] = dataframe['acetohexamide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['glipizide'] = dataframe['glipizide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['glyburide'] = dataframe['glyburide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['tolbutamide'] = dataframe['tolbutamide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['pioglitazone'] = dataframe['pioglitazone'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['rosiglitazone'] = dataframe['rosiglitazone'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['acarbose'] = dataframe['acarbose'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['miglitol'] = dataframe['miglitol'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['troglitazone'] = dataframe['troglitazone'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['tolazamide'] = dataframe['tolazamide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['examide'] = dataframe['examide'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['citoglipton'] = dataframe['citoglipton'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['insulin'] = dataframe['insulin'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['glyburide-metformin'] = dataframe['glyburide-metformin'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['glipizide-metformin'] = dataframe['glipizide-metformin'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['glimepiride-pioglitazone'] = dataframe['glimepiride-pioglitazone'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['metformin-rosiglitazone'] = dataframe['metformin-rosiglitazone'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


dataframe['metformin-pioglitazone'] = dataframe['metformin-pioglitazone'].apply(lambda x: 0 if x == 'no' else (1 if x == 'steady' else (2 if x == 'up' else -1)))


In [13]:
dataframe.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,41,0,...,-1,-1,-1,-1,-1,-1,-1,0,0,2
1,Caucasian,Female,[10-20),1,1,7,3,Unknown,59,0,...,-1,-1,-1,-1,-1,-1,-1,0,0,2
2,AfricanAmerican,Female,[20-30),1,1,7,2,Unknown,11,5,...,-1,-1,-1,-1,-1,-1,-1,0,0,2
3,Caucasian,Male,[30-40),1,1,7,2,Unknown,44,1,...,-1,-1,-1,-1,-1,-1,-1,0,0,2
4,Caucasian,Male,[40-50),1,1,7,1,Unknown,51,0,...,-1,-1,-1,-1,-1,-1,-1,0,0,2


In [14]:

categorical_columns = [
    'race', 'gender', 'age', 'medical_specialty',
    'diag_1', 'diag_2', 'diag_3', 'A1Cresult',
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
    'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
    'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
    'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone',
    'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'
]

existing_columns = [col for col in categorical_columns if col in dataframe.columns]

dataframe = pd.get_dummies(dataframe, columns=existing_columns, dtype=int)

dataframe.head()


,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,insulin_-1,glyburide-metformin_-1,glipizide-metformin_-1,glimepiride-pioglitazone_-1,metformin-rosiglitazone_-1,metformin-pioglitazone_-1,change_0,diabetesMed_0,readmitted_1,readmitted_2
0,6,25,1,1,41,0,1,0,0,0,...,1,1,1,1,1,1,1,1,0,1
1,1,1,7,3,59,0,18,0,0,0,...,1,1,1,1,1,1,1,1,0,1
2,1,1,7,2,11,5,13,2,0,1,...,1,1,1,1,1,1,1,1,0,1
3,1,1,7,2,44,1,16,0,0,0,...,1,1,1,1,1,1,1,1,0,1
4,1,1,7,1,51,0,8,0,0,0,...,1,1,1,1,1,1,1,1,0,1


In [15]:
from sklearn.preprocessing import StandardScaler


numerical_features = [
    'time_in_hospital', 'num_lab_procedures', 'num_procedures',
    'num_medications', 'number_outpatient', 'number_emergency',
    'number_inpatient', 'number_diagnoses'
]


scaler = StandardScaler()
dataframe[numerical_features] = scaler.fit_transform(dataframe[numerical_features])


In [16]:
from sklearn.model_selection import train_test_split

X = dataframe.drop('readmitted_2', axis=1)  
y = dataframe['readmitted_2']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:

print(dataframe.columns)


Index(['admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient',
       ...
       'insulin_-1', 'glyburide-metformin_-1', 'glipizide-metformin_-1',
       'glimepiride-pioglitazone_-1', 'metformin-rosiglitazone_-1',
       'metformin-pioglitazone_-1', 'change_0', 'diabetesMed_0',
       'readmitted_1', 'readmitted_2'],
      dtype='object', length=157)


In [18]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
svm_classifier = SVC(probability=True, random_state=42)
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

In [19]:
hard_voting_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('svm', svm_classifier), ('xgb', xgb_classifier)],
    voting='hard'
)

In [20]:
soft_voting_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('svm', svm_classifier), ('xgb', xgb_classifier)],
    voting='soft'
)

In [21]:
fraction_keep = 0.1
dataframe= dataframe.sample(frac=fraction_keep, random_state=42)
dataframe.reset_index(drop=True, inplace=True)

dataframe.head()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,insulin_-1,glyburide-metformin_-1,glipizide-metformin_-1,glimepiride-pioglitazone_-1,metformin-rosiglitazone_-1,metformin-pioglitazone_-1,change_0,diabetesMed_0,readmitted_1,readmitted_2
0,2,1,7,-0.132659,1.164178,0.973300,0.243376,-0.291463,-0.212621,0.288573,...,1,1,1,1,1,1,1,1,1,0
1,5,1,7,-0.802652,0.249281,-0.785404,-0.125743,2.864942,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,0,1
2,3,1,1,-1.137649,-2.139617,0.387065,0.489455,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,0,1
3,2,1,1,0.202338,-0.258995,-0.785404,-1.356139,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,0,1
4,1,1,7,-1.137649,-1.885479,-0.785404,-1.233100,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0


In [22]:
dataframe.shape

(10176, 157)

In [23]:
X_train.columns = X_train.columns.str.replace(r'[^\w\s]', '_', regex=True)
X_train.head()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,citoglipton__1,insulin__1,glyburide_metformin__1,glipizide_metformin__1,glimepiride_pioglitazone__1,metformin_rosiglitazone__1,metformin_pioglitazone__1,change_0,diabetesMed_0,readmitted_1
41511,1,22,7,-0.802652,-0.258995,0.973300,1.350732,-0.291463,0.862102,1.080425,...,1,1,1,1,1,1,1,1,1,0
24080,6,25,17,0.537334,-1.021409,-0.785404,-0.002703,1.286739,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0
98079,1,1,7,-1.137649,-0.055685,-0.785404,-1.356139,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0
6238,1,1,7,-0.802652,-1.123065,-0.785404,-0.863981,-0.291463,-0.212621,0.288573,...,1,1,1,1,1,1,1,1,1,0
72209,2,1,1,-0.467656,-1.834651,0.973300,-0.125743,0.497638,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0


In [24]:
hard_voting_classifier.fit(X_train, y_train)

C:\Users\ranul\AppData\Roaming\Python\Python310\site-packages\xgboost\core.py:158: UserWarning: [03:51:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svm', SVC(probability=True, random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_ty...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))])

In [25]:
X_test.columns = X_test.columns.str.replace(r'[^\w\s]', '_', regex=True)
X_test.head()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,citoglipton__1,insulin__1,glyburide_metformin__1,glipizide_metformin__1,glimepiride_pioglitazone__1,metformin_rosiglitazone__1,metformin_pioglitazone__1,change_0,diabetesMed_0,readmitted_1
35957,2,1,7,-0.132659,1.164178,0.973300,0.243376,-0.291463,-0.212621,0.288573,...,1,1,1,1,1,1,1,1,1,1
60928,5,1,7,-0.802652,0.249281,-0.785404,-0.125743,2.864942,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0
79921,3,1,1,-1.137649,-2.139617,0.387065,0.489455,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0
50079,2,1,1,0.202338,-0.258995,-0.785404,-1.356139,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,0
44081,1,1,7,-1.137649,-1.885479,-0.785404,-1.233100,-0.291463,-0.212621,-0.503279,...,1,1,1,1,1,1,1,1,1,1


In [26]:
y_pred_hard = hard_voting_classifier.predict(X_test)
print("Hard Voting Accuracy:", accuracy_score(y_test, y_pred_hard))
print("Hard Voting Classification Report:")
print(classification_report(y_test, y_pred_hard))

Hard Voting Accuracy: 1.0
Hard Voting Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2279
           1       1.00      1.00      1.00     18074

    accuracy                           1.00     20353
   macro avg       1.00      1.00      1.00     20353
weighted avg       1.00      1.00      1.00     20353



In [27]:
soft_voting_classifier.fit(X_train, y_train)

C:\Users\ranul\AppData\Roaming\Python\Python310\site-packages\xgboost\core.py:158: UserWarning: [04:06:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svm', SVC(probability=True, random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_ty...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [28]:
y_pred_soft = soft_voting_classifier.predict(X_test)
print("Soft Voting Accuracy:", accuracy_score(y_test, y_pred_soft))
print("Soft Voting Classification Report:")
print(classification_report(y_test, y_pred_soft))

Soft Voting Accuracy: 1.0
Soft Voting Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2279
           1       1.00      1.00      1.00     18074

    accuracy                           1.00     20353
   macro avg       1.00      1.00      1.00     20353
weighted avg       1.00      1.00      1.00     20353

